In [ ]:
import numpy as np
import pandas as pd
from sklearn.ensemble import IsolationForest
from sklearn.model_selection import train_test_split

Leemos el dataset con los datos sin procesar, y hacemos una copia de él, pues lo vamos a manipular.

In [ ]:
df = pd.read_csv("/content/drive/MyDrive/4to año/2do Cuatrimestre/TFG/Materiales/Tráfico IoT/raw_data_from_tshark.csv", delimiter='$')
df.head()
dataset = df
dataset.head()

,frame.number,frame.time,frame.len,eth.src,eth.dst,ip.src,ip.dst,ip.proto,ip.len,tcp.len,tcp.srcport,tcp.dstport,_ws.col.Info
0,1,"Feb 16, 2020 12:37:22.736684743 IST",54,50:02:91:69:66:71,98:22:ef:d5:cc:2f,192.168.0.35,192.168.0.121,6.0,40.0,0.0,49279.0,80.0,"49279 → 80 [FIN, ACK] Seq=1 Ack=1 Win=1910 Len=0"
1,2,"Feb 16, 2020 12:37:22.736773147 IST",62,50:02:91:69:66:71,98:22:ef:d5:cc:2f,192.168.0.35,192.168.0.121,6.0,48.0,0.0,56521.0,80.0,56521 → 80 [SYN] Seq=0 Win=2144 Len=0 MSS=536 ...
2,3,"Feb 16, 2020 12:37:22.736824792 IST",62,98:22:ef:d5:cc:2f,50:02:91:69:66:71,192.168.0.121,192.168.0.35,6.0,48.0,0.0,80.0,56521.0,"80 → 56521 [SYN, ACK] Seq=0 Ack=1 Win=64240 Le..."
3,4,"Feb 16, 2020 12:37:22.736836228 IST",54,98:22:ef:d5:cc:2f,50:02:91:69:66:71,192.168.0.121,192.168.0.35,6.0,40.0,0.0,80.0,49279.0,"80 → 49279 [FIN, ACK] Seq=1 Ack=2 Win=64025 Len=0"
4,5,"Feb 16, 2020 12:37:22.749684991 IST",54,50:02:91:69:66:71,98:22:ef:d5:cc:2f,192.168.0.35,192.168.0.121,6.0,40.0,0.0,56521.0,80.0,56521 → 80 [ACK] Seq=1 Ack=1 Win=2144 Len=0


Eliminamos el momento exacto de la captura y el contenido del frame.

In [ ]:
dataset = dataset.drop(['frame.time'], axis=1)
dataset = dataset.drop(['_ws.col.Info'], axis=1)

Las funciones de a continuación sirven para pasar a entero una dirección mac, y una dirección IP. 

In [ ]:
import re
import ipaddress

def mac_to_int(mac):
    res = re.match('^((?:(?:[0-9a-f]{2}):){5}[0-9a-f]{2})$', mac)
    if res is None:
        raise ValueError('invalid mac address')
    return int(res.group(0).replace(':', ''), 16)

def ip_to_int(ip): 
  try: 
    addr = ipaddress.ip_address(ip)
    return int(addr)
  except:
    return 0

A continuación, lo que hacemos es leer las columnas de eth.src, eth.dst, ip.src, ip.dst e ip.proto para pasarlos a números enteros. 

Como no se pueden modificar los valores del dataset directamente, lo que hacemos es crear un array con los valores que queremos. Y luego sustituimos esa columna con este array. 

In [ ]:
column_mac_src = []
column_mac_dst = []
column_ip_src = []
column_ip_dst = []
column_ip_proto = []
for i in range(len(dataset['ip.src'])):
  aux = str(dataset['eth.src'][i])
  aux = mac_to_int(aux)
  column_mac_src.append(aux)
  aux = str(dataset['eth.dst'][i])
  aux = mac_to_int(aux)
  column_mac_dst.append(aux)
  aux = str(dataset['ip.src'][i])
  aux = ip_to_int(aux)
  column_ip_src.append(aux)
  aux = str(dataset['ip.dst'][i])
  aux = ip_to_int(aux)
  column_ip_dst.append(aux)
  aux = dataset['ip.proto'][i]
  aux = aux.astype(int)
  if aux < 0: aux = 256 # Si es menor que cero es que el valor era nulo. 
  column_ip_proto.append(aux)

dataset['eth.src'] = column_mac_src
dataset['eth.dst'] = column_mac_dst
dataset['ip.src'] = column_ip_src
dataset['ip.dst'] = column_ip_dst
dataset['ip.proto'] = column_ip_proto

Hacemos lo mismo para los campos TCP. 

In [ ]:
column_tcp_len = []
column_tcp_srcport = []
column_tcp_dstport = []

for i in range(len(dataset['tcp.len'])):
  aux = str(dataset['tcp.len'][i])
  #print(aux)
  if aux == 'nan': aux = 0.0
  column_tcp_len.append(aux)

  aux = str(dataset['tcp.srcport'][i])
  if aux == 'nan': aux = 65536
  column_tcp_srcport.append(aux)

  aux = str(dataset['tcp.dstport'][i])
  if aux == 'nan': aux = 65536
  column_tcp_dstport.append(aux)


dataset['tcp.len'] = column_tcp_len
dataset['tcp.srcport'] = column_tcp_srcport
dataset['tcp.dstport'] = column_tcp_dstport

El dataset con los cambios que le hemos hecho:

In [ ]:
dataset.head()

,frame.number,frame.len,eth.src,eth.dst,ip.src,ip.dst,ip.proto,ip.len,tcp.len,tcp.srcport,tcp.dstport
0,1,54,87971959760497,167275820076079,3232235555,3232235641,6,40.0,0.0,49279.0,80.0
1,2,62,87971959760497,167275820076079,3232235555,3232235641,6,48.0,0.0,56521.0,80.0
2,3,62,167275820076079,87971959760497,3232235641,3232235555,6,48.0,0.0,80.0,56521.0
3,4,54,167275820076079,87971959760497,3232235641,3232235555,6,40.0,0.0,80.0,49279.0
4,5,54,87971959760497,167275820076079,3232235555,3232235641,6,40.0,0.0,56521.0,80.0


Aplicamos ML. 

Seleccionamos las variables de nuestro algoritmo no supervisado. 

In [ ]:
data = dataset[['frame.number', 'frame.len', 'eth.src', 'eth.dst', 'ip.src', 'ip.dst', 'ip.proto', 'tcp.len', 'tcp.srcport', 'tcp.dstport']]

El algoritmo seleccionado es el Isolation Forest. 

In [ ]:
isoForest = IsolationForest()
isoForest.fit(data)

IsolationForest(behaviour='deprecated', bootstrap=False, contamination='auto',
                max_features=1.0, max_samples='auto', n_estimators=100,
                n_jobs=None, random_state=None, verbose=0, warm_start=False)

Con el método predict() se devuelve directamente la identificación de anomalía (-1) o no anomalía (1).

In [ ]:
anomaly = isoForest.predict(data)

NotFittedError: ignored

Añadimos la predicción que hemos hecho a nuestro dataset.

In [ ]:
dataset['anomaly'] = anomaly

Mostramos nuestro dataset:

In [ ]:
dataset

,frame.number,frame.len,eth.src,eth.dst,ip.src,ip.dst,ip.proto,ip.len,tcp.len,tcp.srcport,tcp.dstport,anomaly
0,1,54,87971959760497,167275820076079,3232235555,3232235641,6,40.0,0.0,49279.0,80.0,1
1,2,62,87971959760497,167275820076079,3232235555,3232235641,6,48.0,0.0,56521.0,80.0,1
2,3,62,167275820076079,87971959760497,3232235641,3232235555,6,48.0,0.0,80.0,56521.0,1
3,4,54,167275820076079,87971959760497,3232235641,3232235555,6,40.0,0.0,80.0,49279.0,-1
4,5,54,87971959760497,167275820076079,3232235555,3232235641,6,40.0,0.0,56521.0,80.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...
125153,125154,98,167275820076079,110425385261001,3232235641,3232235703,1,84.0,0,65536,65536,-1
125154,125155,98,110425385261001,167275820076079,3232235703,3232235641,1,84.0,0,65536,65536,-1
125155,125156,98,167275820076079,110425385261001,3232235641,3232235703,1,84.0,0,65536,65536,-1
125156,125157,98,110425385261001,167275820076079,3232235703,3232235641,1,84.0,0,65536,65536,-1
